In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
!pip install transformers

In [3]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import BertConfig

In [4]:
!pip install -U spacy[cuda92]
!python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm
spacy.prefer_gpu()
spacy_nlp = en_core_web_sm.load()

Requirement already up-to-date: spacy[cuda92] in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages (2.3.2)
ERROR: Could not find a version that satisfies the requirement cupy-cuda92<9.0.0,>=5.0.0b4; extra == "cuda92" (from spacy[cuda92]) (from versions: none)
ERROR: No matching distribution found for cupy-cuda92<9.0.0,>=5.0.0b4; extra == "cuda92" (from spacy[cuda92])
/usr/bin/python: No module named spacy


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [6]:
DIR = "question_generator/"
PRETRAINED_MODEL = 'bert-base-cased'
BATCH_SIZE = 16
SEQ_LENGTH = 512

tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL)

class QAEvalDataset(Dataset):
    def __init__(self, csv):
        self.df = pd.read_csv(csv, engine='python')
        self.transforms = [self.shuffle, self.corrupt]

    def __len__(self):
         return len(self.df)

    def __getitem__(self, idx): 
        _, question, answer = self.df.iloc[idx]
        label = random.choice([0, 1])

        if label == 0:
            question, answer = random.choice(self.transforms)(question, answer)

        encoded_data = tokenizer(
            text=question,
            text_pair=answer,
            pad_to_max_length=True, 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )

        encoded_data['input_ids'] = torch.squeeze(encoded_data['input_ids'])
        encoded_data['token_type_ids'] = torch.squeeze(encoded_data['token_type_ids'])
        encoded_data['attention_mask'] = torch.squeeze(encoded_data['attention_mask'])
        return (encoded_data.to(device), torch.tensor(label).to(device))
    
    def shuffle(self, question, answer):
        shuffled_answer = answer
        while shuffled_answer == answer:
            shuffled_answer = self.df.sample(1)['answer'].item()
        return question, shuffled_answer
    
    def corrupt(self, question, answer):
        doc = spacy_nlp(question)
        if len(doc.ents) > 1:
            # Replace all entities in the sentence with the same thing
            copy_ent = str(random.choice(doc.ents))
            for ent in doc.ents:
                question = question.replace(str(ent), copy_ent)
        elif len(doc.ents) == 1:
            # Replace the answer with an entity from the question
            answer = str(doc.ents[0])
        else:
            question, answer = self.shuffle(question, answer)
        return question, answer


train_set = QAEvalDataset(os.path.join(DIR, 'qa_eval_train.csv')) 
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valid_set = QAEvalDataset(os.path.join(DIR, 'qa_eval_valid.csv')) 
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
LR = 0.001
EPOCHS = 10
LOG_INTERVAL = 500

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
SAVED_MODEL_PATH = "question_generator/qa_eval_model_trained.pth"

def train():
    model.train()
    total_loss = 0.
    for batch_index, batch in enumerate(train_loader):
        data, labels = batch
        optimizer.zero_grad()
        output = model(**data, labels=labels)
        loss = output[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()
        
        if batch_index % LOG_INTERVAL == 0 and batch_index > 0:
            cur_loss = total_loss / LOG_INTERVAL
            print('| epoch {:3d} | ' 
                  '{:5d}/{:5d} batches | '
                  'loss {:5.2f}'.format(
                    epoch, 
                    batch_index, len(train_loader), 
                    cur_loss))
            total_loss = 0

def evaluate(eval_model, data_loader):
    eval_model.eval()
    total_score = 0.
    with torch.no_grad():
        for batch_index, batch in enumerate(data_loader):
            data, labels = batch
            output = eval_model(**data, labels=labels)
            preds = np.argmax(output[1].cpu(), axis=1)
            total_score += (preds == labels.cpu()).sum()
    return total_score / (len(data_loader) * BATCH_SIZE)

def save(epoch, model_state_dict, optimizer_state_dict, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'best_loss': loss,
            }, SAVED_MODEL_PATH)

    print("| Model saved.")
    print_line()

def load():
    return torch.load(SAVED_MODEL_PATH)

def print_line():
    LINE_WIDTH = 60
    print('-' * LINE_WIDTH)

In [ ]:
highest_accuracy = 0

accuracy = evaluate(model, valid_loader)
print_line()
print('| Before training | accuracy on valid set: {:5.2f}%'.format(accuracy))
print_line()

for epoch in range(1, EPOCHS + 1):

    train()
    accuracy = evaluate(model, valid_loader)
    print_line()
    print('| end of epoch {:3d} | accuracy on valid set: {:5.2f}%'.format(
        epoch,
        accuracy)
    )
    print_line()

    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        save(
             epoch, 
             model.state_dict(), 
             optimizer.state_dict(), 
             highest_accuracy
        )